In [152]:
import pandas as pd
import numpy as np
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from geopy.geocoders import Nominatim


In [153]:
df = pd.read_csv('../data/df_with_lat_long.csv')

In [154]:
df

,Neighbourhood,City,Country,Latitude,Longitude
0,Charlottenburg-Wilmersdorf,Berlin,Germany,52.507856,13.263952
1,Friedrichshain-Kreuzberg,Berlin,Germany,52.501115,13.444285
2,Lichtenberg,Berlin,Germany,52.532161,13.511893
3,Marzahn-Hellersdorf,Berlin,Germany,52.522523,13.587663
4,Mitte,Berlin,Germany,52.517885,13.404060
...,...,...,...,...,...
157,READ BLVD EAST,New Orleans,United States,30.031682,-89.975295
158,WEST LAKE FOREST,New Orleans,United States,42.221968,-87.873126
159,LITTLE WOODS,New Orleans,United States,30.075200,-89.944235
160,FAIRGROUNDS,New Orleans,United States,29.983767,-90.081270


In [155]:
def get_lat_long(address):
    geolocator = Nominatim(user_agent="Get_Geo_Codes")
    location = geolocator.geocode(address)
    if location is not None:
        lat_ =location.latitude
        long_ = location.longitude
    else:
        lat_ =long_ = None
        
    return lat_,long_
    

In [156]:
cities = list(df['City'].unique())
cities

['Berlin', 'Alberta', 'New Orleans']

In [157]:
city_point={}
arr=[]
for city in cities:
    lat_ , long_ =get_lat_long(city)
    city_point['City']=city ;city_point['lat']=lat_ ;city_point['long']=long_
    arr.append(city_point)
    city_point={}

In [158]:
arr

[{'City': 'Berlin', 'lat': 52.5170365, 'long': 13.3888599},
 {'City': 'Alberta', 'lat': 55.001251, 'long': -115.002136},
 {'City': 'New Orleans', 'lat': 29.9759983, 'long': -90.0782127}]

In [159]:
df_city=pd.DataFrame(arr)
df_city

,City,lat,long
0,Berlin,52.517037,13.388860
1,Alberta,55.001251,-115.002136
2,New Orleans,29.975998,-90.078213


In [160]:
city_map = folium.Map(location = [0,0], zoom_start = 2)

for lat, lng, label in zip(df_city['lat'], df_city['long'], df_city['City']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(city_map) 
    
city_map

In [161]:
def generate_map (city, zoom):

    map = folium.Map(location=[df_city.loc[df_city['City']==city,'lat'].values[0],df_city.loc[df_city['City']==city,'long'].values[0]], zoom_start=zoom)
    #add markers to the cities
    for lat, lng, label in zip(df.loc[df['City']==city,'Latitude'], df.loc[df['City']==city,'Longitude'], df.loc[df['City']==city, 'Neighbourhood']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7).add_to(map)

    return map

In [162]:
map_berlin = generate_map('Berlin', 8)
map_berlin

In [163]:
map_alberta = generate_map('Alberta', 3)
map_alberta

## Outlaier in Canada

In [164]:
more_50 = df.loc[(df['Country']=='Canada') & (df['Latitude']<50)]
canada_outlaier =more_50.loc[(more_50['Country']=='Canada') & (more_50['Longitude']<-77)]
added_to_out= df.loc[df.Neighbourhood.isin(['West QE2','Anders South','Riverside Meadows','Johnstone Crossing','Michener Hill'])]
outs=pd.concat([canada_outlaier,added_to_out])

In [165]:
df.drop(index=outs.index,inplace=True)

In [166]:
map_alberta = generate_map('Alberta', 3)
map_alberta

In [167]:
map_alberta = generate_map('New Orleans', 3)
map_alberta

## Outlaier in US

In [168]:
out=df.loc[df.Neighbourhood.isin(['VILLAGE DE LEST','ST. THOMAS DEV','ST. BERNARD AREA'])]

In [169]:
df.drop(index=out.index,inplace=True)

In [170]:
map_alberta = generate_map('New Orleans', 3)
map_alberta